In [ ]:
# !pip install langchain langchain-community langchain-text-splitters langchain-huggingface langchain-chroma pypdf transformers torch accerlate timm

  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_huggingface-0.3.0-py3-none-any.whl.metadata (996 bytes)
  Using cached langchain_chroma-0.2.4-py3-none-any.whl.metadata (1.1 kB)
  Using cached pypdf-5.7.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached torch-2.7.1-cp310-cp310-win_amd64.whl.metadata (28 kB)
  Using cached langchain_core-0.3.67-py3-none-any.whl.metadata (5.8 kB)
  Using cached langsmith-0.4.4-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.41-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.embeddings import Embeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModel
import torch
from typing import List

from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_chroma import Chroma

from langchain.chains import create_retrieval_chain

c:\Users\user\miniconda3\envs\llm_p_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPU 확인

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용할 장치: {device}")
if device == "cuda":
    print(f"사용 가능한 GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU 장치: {torch.cuda.get_device_name(0)}")
print("-" * 50)

사용할 장치: cuda
사용 가능한 GPU 개수: 1
현재 GPU 장치: NVIDIA GeForce RTX 4090
--------------------------------------------------


## 1. 데이터 준비 (로더, 스플릿터)

In [ ]:
print("[1단계] PDF 데이터 로딩 및 분할 시작")

pdf_path = r'C:\Users\user\Documents\workspace_llm\workspace_llm_p_2\기관사3급_ocr.pdf'

if not os.path.exists(pdf_path):
    print(f"오류: PDF 파일을 찾을 수 없습니다. 경로를 확인하세요: {pdf_path}")
    exit()

loader = PyPDFLoader(pdf_path)
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)
print(f"PDF 로딩 및 분할 완료. 총 분할된 문서 조각 수: {len(all_splits)}")
print("-" * 50)

[1단계] PDF 데이터 로딩 및 분할 시작
PDF 로딩 및 분할 완료. 총 분할된 문서 조각 수: 1038
--------------------------------------------------


## 2. 임베딩 모델 (커스텀 클래스)

In [ ]:
model_name = "BAAI/bge-m3"

embedding_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True} # 벡터 정규화
)
print("벡터정규화 (임베딩) 준비 완료")

벡터정규화 (임베딩) 준비 완료


## 3. 벡터 db에 저장 (임베딩, 리트리버 설정)

In [ ]:
print("[3단계] 벡터 DB 생성 및 Retriever 설정 시작")


persist_directory = './chroma_db_gemma3_n_test_v1'

if not os.path.exists(persist_directory):
    print("새로운 Chroma 벡터 스토어를 생성합니다. (시간이 소요될 수 있습니다...)")
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embedding_function,
        persist_directory=persist_directory
    )
    print("✅ 벡터 스토어 생성 및 저장 완료.")
else:
    print("기존 Chroma 벡터 스토어를 로드합니다.")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_function
    )
    print("✅ 벡터 스토어 로드 완료.")

retriever = vectorstore.as_retriever(search_kwargs={"k": 7})
print("✅ Retriever 설정 완료. (상위 7개 문서 검색)")
print("-" * 50)

[3단계] 벡터 DB 생성 및 Retriever 설정 시작
기존 Chroma 벡터 스토어를 로드합니다.
✅ 벡터 스토어 로드 완료.
✅ Retriever 설정 완료. (상위 7개 문서 검색)
--------------------------------------------------


## llm모델 지정

In [ ]:
print("[4단계] LLM 준비 시작")

llm_model_id = "google/gemma-3n-E2B-it"

tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager" # triton 설정 : 가장 기본적인 pytorch 어텐션 구현
)

pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.2, 
    return_full_text=False 
)

llm = HuggingFacePipeline(pipeline=pipe)
print("✅ LLM 준비 완료 : {llm_model_id}")
print("-" * 50)

[4단계] LLM 준비 시작


Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.10s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


✅ LLM 준비 완료 : {llm_model_id}
--------------------------------------------------


## 프롬프트 및 RAG Chain 생성


In [ ]:
print("[5단계] RAG 체인 생성 시작")

prompt = ChatPromptTemplate.from_template("""<start_of_turn>user
You are a Q&A bot that must answer the user's “Question” using only the provided “Retrieved Documents.”

**Your mission:**
1. Fully comprehend and analyze the content of the “Retrieved Documents.”
2. Never use your prior knowledge or any external information. You must base your answers solely on what is explicitly stated in the “Retrieved Documents.”
3. If the answer to the question clearly exists within the “Retrieved Documents,” locate it and provide the correct answer along with a detailed explanation.
4. If you cannot find the answer to the question within the “Retrieved Documents,” do not guess—honestly respond, “I cannot answer based on the provided documents.”


**think step-by-step inside a <thinking> block to analyze the problem:**
1.  Identify the core question the user is asking.
2.  Scan the 'Context' to find the multiple-choice question and its options that match the user's query.
3.  Locate any text in the 'Context' that serves as an explanation, a definition, or a clue to the correct answer.
4.  Determine the correct option based on the evidence found.
5.  Formulate a clear explanation based on your findings.

After your thinking process, generate the final user-facing answer in Korean, strictly following the format below.

## Final Answer Format:
**Final Answer :**
[정답 보기와 내용을 명시합니다. 예: 사. 고속선에서는 경사가 작은 추진기를 사용한다.]

**해설:**
[찾아낸 근거를 바탕으로 정답인 이유와 오답인 이유를 상세하게 설명합니다. 예: 프로펠러 날개를 후방경사 하면 추진기의 효율은 증가 한다.]

---
**IMPORTANT**: If the provided 'Context' is insufficient to determine the correct answer and provide a clear explanation, your final answer must be: "제공된 정보만으로는 질문에 대한 정확한 해설이나 정답을 찾을 수 없습니다." Do not use any external knowledge.

# Context:
{context}

# Question:
{input}<end_of_turn>
<start_of_turn>model
<thinking>
1. Identify the core question.
2. Locate the matching question and options in the Context.
3. Find evidence (definitions, explanations, clues) in the Context.
4. Select the correct option based on evidence.
5. Prepare a clear explanation.
</thinking>
""")

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)
print("✅ RAG 체인 생성 완료.")
print("-" * 50)

[5단계] RAG 체인 생성 시작
✅ RAG 체인 생성 완료.
--------------------------------------------------


## 실행 (invoke)

In [ ]:
print("[6단계] RAG 체인 실행")
question = "해양환경관리법에 의한 기름오염방지설비 중 기름받이의 기준으로 옳지 않은 것은? 가. 재질은 금속제일 것 나. 선박이 세로방향으로 10도 또는 가로방향으로 22.5도 경사하는 경우에도 기름이 넘치거나 흐르지 않는 구조일 것 사. 기름이 새어나올 우려가 있는 각종 기기나 탱크(tank)의 밑에 설치 아. 모든 탱크의 에어벤트(tank air vent)에는 반드시 기름 받이가 있을 것"

print(f"💬 질문: {question}\n")

response = rag_chain.invoke({"input": question})

print("--- [검색된 문서 (Context)] ---")
for i, doc in enumerate(response["context"]):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    
print("-" * 30)
print("--- [최종 답변] ---")
print(response["answer"])

[6단계] RAG 체인 실행
💬 질문: 해양환경관리법에 의한 기름오염방지설비 중 기름받이의 기준으로 옳지 않은 것은? 가. 재질은 금속제일 것 나. 선박이 세로방향으로 10도 또는 가로방향으로 22.5도 경사하는 경우에도 기름이 넘치거나 흐르지 않는 구조일 것 사. 기름이 새어나올 우려가 있는 각종 기기나 탱크(tank)의 밑에 설치 아. 모든 탱크의 에어벤트(tank air vent)에는 반드시 기름 받이가 있을 것

--- [검색된 문서 (Context)] ---
문서 1: 비 중 기름받이의 기준으로 옳지 않은 것은? 
가. 재질은 금속제일 것
나. 선박이 세로방향으로 10도 또는 가로방향 
으로 22.5도 경사하는 경우에도 기름이 
넘치거나 흐르지 않는 구조일 것
사. 기름이 새어나올 우려가 있는 각종 기기 
나 탱크(tank)의 밑에 ...
문서 2: 직무일반 척중 예상 묜졔
가 수면위로 올라올 가능성이 있어 선미를 무겁게 
하기 위해 배수하여서는 안된다.
223 ► 연료유 저장탱크의 점검차 탱크 내부에 
들어가는 경우의 주의사항 중 옳지 않은 것은?
가. 송풍장치로 탱크 내부의 가스를 제거한 
후 들어갈 것
나. ...
문서 3: 217 ► 다음 중 기름의 해양배출이 허용되는 경 
우는?
가. 항해 중 15[ppm] 이상의 유분농도의 
처리수로 배출할 경우
나. 영해기선으로부터 25해리 이상에서 항 
해 중에 기관실 빌지펌프를 통해 배출하 
는 경우
사. 기관실 기기를 수리한 후 발생한 폐유를 ...
문서 4: s
아. 기름 탱크 속에서 작업을 할 때에는 미리 
충분히 환기시켜야 한다.
/ 몸은 전류가 흐르는 도체 이므로 옷을 길게 입어 
전기가 통하지 않도록 한다.
162 ► 다음 중 기름 또는 유해액체물질의 해양 
오염비상계획서가 비치되어야 할 선박이 아 
닌 것은?
가....
문서 5: 나. 기름유출사고 발생 시 해당 선박의 기관장 
이 취해야 할 보고 절차에 관한 사항
人1. 선원의 방제교육 • 훈련에 관한 사항 
아. 기름유출을 줄이기 위하여

1장-1번 한국어 문제, 정답(26초), 리트리버 O, 해설 O

[6단계] RAG 체인 실행
💬 질문: 디젤기관에서 연료소비율이란? 가. 기관이 1시간에 소비하는 연료량 나. 연료의 시간당 발열량 사. 기관이 1시간당 1마력을 얻기 위해 소비하는 연료량 아. 기관이 1실런더당 1시간에 소비하는 연료량

--- [검색된 문서 (Context)] ---
문서 1: PART
01적중 예상 문제
001 ► 디젤기관에서 연료소비율이란?
가. 기관이 1시간에 소비하는 연료량
나. 연료의 시간당 발열량
사. 기관이 1시간당 1마력을 얻기 위해 소비하 
는 연료량
아. 기관이 1실린더당 1시간에 소비하는 연료량
/ 연료소비율: 연료소비율은...
문서 2: 배기터빈 효율이 높다.
019 ► 디젤기관의 고출력화를 위한 방법 중 
비출력을 증대시키는 방법이 아닌 것은?
가. 행정용적을 크게 한다.
나. 회전수를 높인다.
사. 평균유효압력을 높인다.
아. 과급기의 효율을 높인다.
/ 비출력: 동력 기계어서 연료가 단위량 당 생...
문서 3: 기관 I 적중 예상 묜졔
가. 점도지수 나. 비점도
사. 절대점도 아. 동점도
/ 동점도 : 점성도를 밀도로 나눈 값
130 ► 다음 중 저속 디젤기관에서 가장 많이 
사용되는 연소실 형태는?
가. 직접분사식 연소실
나. 예연소실식 연소실 
사. 와실식 연소실 
아. ...
문서 4: 04. 내연기관의 열효율에 대한 설명으로 옳은 것 
은?
가. 최고압력이 일정할 때는 오토 사이클의 
효율이 가장 좋다.
나. 최고압력이 일정할 때는 디젤 사이클의 
효율이 가장 좋다.
사. 최고압력이 일정할 때는 사바데 사이클의 
효율이 가장 좋다.
아. 최고압력이 ...
문서 5: 가■. A : mechanical efficiency,
B : specific fuel oil consumption 
나. A : thermal efficiency,
B : specific fuel oil consumption 
시’. A : mechanical eff...
문서 6: 기관 I 적중 예상 문제
가. 세탄가가 낮은 기름을 사용한다.
나. 압축비를 증가시킨다.
사. 착화시까지 연료 분사량을 적게 한다. 
아. 실린더의 냉각수 온도를 높인다.
/ 세탄가 높은 기름을 사용하여야 한다.
062 ► 디젤기관에서 밸브 태핏 간격을 주는 이유 
는...
문서 7: 제 2 회 [ 기관 I ]
01. 디젤기관에서 체적효율의 저하 원인으로 옳지 
않은것은?
가. 흡입공기의 온도가 높을 때 
나. 실린더 내의 온도가 높을 때 
사. 흡■배기밸브의 개폐시기가 적당하지 않 
을 때
아. 실린더의 배압이 낮을 때
02. 내연기관의 이론 공기...
------------------------------
--- [최종 답변] ---
## Final Answer Format:
**Final Answer :** 사. 기관이 1시간당 1마력을 얻기 위해 소비하는 연료량

**해설:**
질문은 디젤기관에서 연료소비율이 무엇인지 묻고 있습니다. Context에서 "연료소비율: 연료소비율은 일정한 일을 하기 위해 엔진이 얼마나 많은 연료를 사용해야 하는가를 나타내는 수치"라고 정의하고 있습니다. 따라서, 기관이 1시간당 1마력을 얻기 위해 소비하는 연료량은 연료소비율에 해당합니다.

---


1장-28번. 한국어 문제 + 오타 포함, 오답(32초) 리트리버 O, 해설 X

[6단계] RAG 체인 실행
💬 질문: 금속면의 눌어붙음 및 마멸을 방지하며 고하중이 걸리는 기어 등을 윤활하는 윤활유에 사용하는 첨가제는? 가. 유동점강하제 나. 산화방지제 사. 청정분산제 아. 극압첨가제제

--- [검색된 문서 (Context)] ---
문서 1: / 파라핀계 물질은 반응성이 약하고 화학약품에 
대하여 내성이 있다.
028 ► 금속면의 눌어붙음 및 마멸을 방지하 
며 고하중이 걸리는 기어 등을 윤활하는 윤활유 
에 사용하는 첨가제는?
가. 유동점강하제 나. 산화방지제
사. 청정분산제 아. 극압첨가제
/ 극압첨가제...
문서 2: 준다.
/ 유성향상제: 금속 표면에 첨가제가 흡착된 막을 
이루어, 경계에 윤활할 경우 유막이 끊어지지 
않게 하고 마찰 계수를 작게 해 준다
173 ► 디젤기관의 발화늦음과 직접적으로 관계 
가 없는 것은?
170 ► 시마진(sea margin)올 옳게 설명한 
것은...
문서 3: t
TBN값이 큰 윤활유를 사용한다.
아. 피스톤링과 실린더 라이너의 길들이기 운 
전시에는 TBNO| 가장 높은 윤활유를 사 
용하는 것이 바람직하다.
25. 윤활유를 열화시키는 원인으로 옳지 않은 것 
은?
가. 이물질 혼입
나. 수분 혼입
사. 산소에 의한 산화
...
문서 4: 가. 산소에 의한 산화
나. 열에 의한 탄화
사. 수분에 의한 유화
아. 신유에 의한 희석
/ 윤활유의 열화원인
1. 산소에 의한 산화
2. 물이나 금속 가루의 혼입
3. 연소 생성물의 혼입
4. 열을 받거나 압력을 받을 경우
058 ► 다음 중 디젤기관의 기계 손실에...
문서 5: J 아WTER 01 유쳬기재 및 환경오염 방지기기 P^ I 기관II
나. 관의 지름을 크게 한다.
다. 관 내의 유속을 낮게 한다.
라. 밸브를 펌프의 송출구 가까이에 설치한다.
S 청정기
윤활유, 연료유를 장시간 사용하게 되면 각종 불순물과 수분 등이 혼입될 수 있 ...
문서 6: 최신 문제 04화
23. 유성에 대한 설명으로 옳지 않은 것은?
가. 일정 조건하에서 서로 다른 종류의 윤활 
유를 비교하는 경우 점도 차이만으로 설 
명할 수 없는 마찰력 차이의 원인이 되는 
기름의 성질이다.
나. 광물성유는 일반적으로 동•식물성유에 
비해서 유성이...
문서 7: 함으로써 급수를 저장 할 수 았고, 공기를 분리하 
여 용존산소를 제거할 수 있으나 증기의 응축은 
증기 응축기의 역할이다.
094 ► 프러폘러의 개비테이션으로 인하여 나타 
나는 현상은?
가. 엔진 회전수 저하
나. 프로펠러 축 변형
사. 선속 증가
아. 프로펠러 날...
------------------------------
--- [최종 답변] ---
## Final Answer:
**Final Answer:** 사. 청정분산제
**해설:** 질문은 금속면의 눌어붙음 및 마멸을 방지하며 고하중이 걸리는 기어 등을 윤활하는 윤활유에 사용하는 첨가제에 대한 것입니다. Context에서 "극압첨가제"는 기계의 마찰면에 큰 압력이 걸려 유막이 파괴되기 쉬운 윤활상태에 대응하기 위해 윤활유에 첨가된다고 설명하고 있습니다. 반면, "청정분산제"는 윤활유에 포함된 불순물을 분산시켜 윤활유의 성능을 유지하는 역할을 합니다. 따라서, 금속면의 눌어붙음 및 마멸을 방지하고 고하중이 걸리는 기어 등을 윤활하는 윤활유에 사용하는 첨가제는 "청정분산제"입니다.

---



1장-100번. 한국어 문제, 정답(52초) 리트리버 O, 해설 X

[6단계] RAG 체인 실행
💬 질문: 다음 중 디젤기관의 윤활유가 열화하는 원인으로 틀린 것은? 가. 불완전 연소에 의해 생긴 연소 잔사물의 혼입 나. 연료유가 혼입되었을 때 사. 첨사제 사용으로 중성이 되었을 때 아. 불 용해성 물질이 혼입되었을 때때

--- [검색된 문서 (Context)] ---
문서 1: 가. 산소에 의한 산화
나. 열에 의한 탄화
사. 수분에 의한 유화
아. 신유에 의한 희석
/ 윤활유의 열화원인
1. 산소에 의한 산화
2. 물이나 금속 가루의 혼입
3. 연소 생성물의 혼입
4. 열을 받거나 압력을 받을 경우
058 ► 다음 중 디젤기관의 기계 손실에...
문서 2: 흐
082 ► 윤활유의 산화촉진과 무관한 것은?
가. 신유 혼입 나. 사용유 혼입
사. 수분 첨가 아. 유온 상승
/ 신유를 혼입하면 가존 윤활유의 산도보다 오히려 
낮기 때문에 약간 중화 될 수 있다.
083 ► 유성(Oiliness)의 설명으로 틀린 것은?
가. 윤...
문서 3: 아. 연료유의 점성이 작을수록 유립은 작게 
된다.
08. 연접봉 대단부 베어링의 조립에서 베어링 셀 
을 베어링 브라스 단면보다 튀어 나오게 하는 
주된 이유는?
가. 베어링의 과열을 방지하기 위하여
나. 하중에 의한 변형을 방지하기 위하여 
사. 베어링 볼트를 보호...
문서 4: 기관 I 적중 예상 문졔
가. 탄소분 나. 유황분
사. 회분 아. 바나듐분
/ 저온부식의 원인은 주로 황성분 때문이다.
181 ► 디젤기관이 시동공기로 회전은 하지만 착 
화되지 않는 경우들이다. 다음 중 관계 없는 항목 
은?
가. 시동공기압력이 너무 높을 때
나. ...
문서 5: 있으면 연료분사펌프와 노즐의 고장 원 
인이 된다.
아. 유황이 연소되어 H2SO4가 형성되고 이 
것이 고온부식의 직접적인 원인이 되기 
때문에 유황이 되도록 적은 것이 바람직 
하다.
23•중유의 원소 중유효 성분은?
가. 탄소와 질소 나. 탄소와 수소
사. 질소와...
문서 6: 기관 I 적중 예상 몬졔
034 ► 디젤기관이 시동 공기로는 잘 돌지만 
폭발하지 않을 때의 원인과 관계없는 것은?
가. 압축 압력의 저하
나. 연료밸브 노즐의 막힘
사. 연료유의 분사시기가 부적당할 때
아. 시동공기 관제밸브의 고착
/ 기관이 시동공기로는 회전되지만 ...
문서 7: CHAPTER 01. 내연기관 |.arE 1. 기관 I
나. 크랭크축심의 부정
다. 선체 또는 기관대의 변형과 굽힘
라. 윤활유의 부족
마. 메인 베어링의 조정불량
바. 과부하운전
0 디젤기관의 연소
1) 디젤기관의 연소기간
(1) 착화지 연기간
연료가 분사 되어도 자...
------------------------------
--- [최종 답변] ---
## Final Answer Format:
**Final Answer :** 사. 첨사제 사용으로 중성이 되었을 때

**해설:** 윤활유의 열화 원인은 다음과 같습니다.
가. 불완전 연소에 의해 생긴 연소 잔사물의 혼입: 연소 잔사물은 윤활유에 혼입되어 윤활유의 점성을 변화시키고, 금속 표면에 부식성 물질을 공급하여 윤활유의 열화를 가속화합니다.
나. 연료유가 혼입되었을 때: 연료유는 윤활유의 점성을 변화시키고, 금속 표면에 부식성 물질을 공급하여 윤활유의 열화를 가속화합니다.
사. 불 용해성 물질이 혼입되었을 때: 불 용해성 물질은 윤활유에 혼입되어 윤활유의 점성을 변화시키고, 금속 표면에 부식성 물질을 공급하여 윤활유의 열화를 가속화합니다.
아. 불 용해성 물질이 혼입되었을 때: 불 용해성 물질은 윤활유에 혼입되어 윤활유의 점성을 변화시키고, 금속 표면에 부식성 물질을 공급하여 윤활유의 열화를 가속화합니다.

첨사제 사용으로 윤활유가 중성이 되는 것은 열화 원인이 아닙니다. 첨사제는 윤활유의 특성을 개선하기 위해 첨가되는 물질로, 윤활유의 열화를 유발하는 원인이 아닙니다.

---

**Final Answer :** 사. 첨사제 사용으로 중성이 되었을 때

**해설:** 윤활유의 열화 원인은 불완전 연소에 의한 연소 잔사물 혼입, 연료유 혼입, 불용해성 물질 혼입 등입니다. 첨사제는 윤활유의 특성을 개선하기 위해 첨가되는 물질로, 윤활유의 열화를 유발하는 원인이 아닙니다. 따라서 첨사제 사용으로 윤활유가 중성이 되는 것은 열화 원인이 아닙니다.


2장-252번. 한국어 문제, 오답(25초) 리트리버 O, 해설 x

[6단계] RAG 체인 실행
💬 질문: 하나의 측정기구로 외경, 내경, 깊이를 모두 측정할 수 있는 것은은? 가. 버니어 캘리퍼스 나. 다이얼 게이지 사. 마이크로 미터 아. 필러 게이지

--- [검색된 문서 (Context)] ---
문서 1: 기관II 적중 예상 묜졔
252 ► 하나의 측정기구로 외경, 내경, 깊이를 모두 
측정할 수 있는 것은?
가. 버니어 캘리퍼스 나. 다이얼 게이지 
사. 마이크로미터 아. 필러 게이지
/ 버니어 캘리퍼스: 원형으로 된 것의 자름, 원통의 
안지름 등을 측정하는 데 주로...
문서 2: 2) 마이크로 미터: U자형 프레임 한쪽은 앤빌이 있고 다른 끝은 슬리브 안쪽은 암나 
사로 되어 이것을 회전시키면 스핀들이 축방향으로 이동하여 측정물을 측정한다.
마이(이티 누증 外 방법
3) 다이얼 게이지
측정하려는 부분에 측정자를 대어 스핀들의 미세한 움직임을 기...
문서 3: 기관 n 적중 예상 문졔
178 ► 버니어 캘리퍼스로 측정할 수 없는 것 
은?
가. 내경 나. 외경
사. 나사유효경 아. 깊이
/ 버니어 캘리퍼스: 원형으로 된 것의 자름, 원통의 
안지름 등을 측정하는 데 주로 사용된다. 본척과 
본척 위를 아동하는 버니어로 되어 ...
문서 4: 섕크 끝에 절삭날을 갖는 절삭 공구
110 ► Rack와 Pinion을 아용하여 마소 길아를 확 
대 표시하는 기구로 회전축의 흔들림 점검, 공작 
물의 평행상태를 측정할 때 사용하는 기구는?
7}. 표준 gauge
나. Dial gauge
사'. Vernier Cal...
문서 5: 최신 문제 03회
18. 메거(Megger)의 주된 용도는?
가. 주파수 계측
나. 접지용 전선의 저항 계측 
사. Fuse 불량 여부 계측 
아. 절연저항 계측
19. 버니어 캘리퍼스의 본척(어미자) 19[mm]를 
20등분한 부척(아들자)의 1눈금 길이는 몇 
[m...
문서 6: 1
°}. fuel injection valves
/ 주기관의 에어러닝을 하기 위해 모든 실린더의 
테스트 콕을 열었다.
287 ► Nearly all fires can be easily 
controlled by one man with a portable ( 
).
...
문서 7: 사. 노즐과 플래퍼 아. 대시포트
/ 압력에 따라 다이어프램의 산축을 이용하여 변위 
를 검출한다.
021 ► 영구자석의 자계 내에 코일을 두고 여 
기에 직류전류를 통과시켜 발생되는 구동력으 
로 작동하는 측정 장치는?
가. 가동코일형 계기
나. 가동철편형 계기
사....
------------------------------
--- [최종 답변] ---
## Final Answer Format:
**Final Answer :** 사. 마이크로미터

**해설:**
질문은 하나의 측정기구로 외경, 내경, 깊이를 모두 측정할 수 있는 측정기구를 묻고 있습니다. Context에서 버니어 캘리퍼스, 다이얼 게이지, 마이크로미터, 필러 게이지에 대한 설명이 제공되어 있습니다. 버니어 캘리퍼스는 원형으로 된 것의 자름, 원통의 안지름 등을 측정하는 데 주로 사용되며, 마이크로미터는 U자형 프레임 한쪽은 앤빌이 있고 다른 끝은 슬리브 안쪽은 암나사로 되어 이것을 회전시키면 스핀들이 축방향으로 이동하여 측정물을 측정하는 측정기입니다. 따라서 마이크로미터는 외경, 내경, 깊이를 모두 측정할 수 있는 측정기구입니다.

---


2장-120번. 숫자 계산 문제. 정답(24초), 리트리버 O, 해설 O

[6단계] RAG 체인 실행
💬 질문: 1냉동톤은 0도의 순수한 물 1톤을 0도의 얼음으로 바꾸는 데 소요되는 시간이 얼마일 때의 냉동능력인가? 가. 1시간 나. 12시간 사. 24시간 아. 48시간간

--- [검색된 문서 (Context)] ---
문서 1: 나. 냉매가스의 비용적 증가 
사. 냉동능력 저하 
아. 증발기 내 증발압력 상승
/ 냉동유는 증발기 전열면에 달라붙어 열교환을 
방해하여 냉동능력을 떨어뜨리나, 증발압력의 상 
승과는 거리가 멀다.
120 ► 1 냉동톤은 o【℃]의순수한 물 1 톤을 이。c] 
의 얼...
문서 2: 최신 문제 04 회
에 Q,【kcal]의 열량을 방출하는 이론 냉동사 
이클의 성적계수는?
가. QJ ( Q, - Q0
나. QJ ( 0. - Q2)
사. QJ ( Q, + QJ
아. QJ ( Q.+ QJ
17. 5[PS]로 한시간 동안 한 일은 약 몇 [kcal]인
...
문서 3: 최신 문제 07희
아. 작동유의 유량 측정
06• 증기 압축식 냉동기에 사용되는 직접냉매가 
갖추어야 할 조건 중 응축압력과 응고온도에 
대한 특성으로 옳은 것은?
가. 응축압력과 응고온도는 모두 높아야 핸다. 
나. 응축압력은 높고 응고온도는 낮아야 한다. 
사. 응...
문서 4: 기관!] 적중 예상 문졦
나. 흡입관의 직경을 크게 하고, 흡입측의 
압력손실을 작게 한다.
사. 펌프의 회전속도를 높게 한다.
아. 회전차 내의 유속을 낮게 한다.
/ 캐비테이션은 수중에서 임펠러의 회전이 발라지 
면 발생하므로 펌프의 회전속도를 낮춰야 한다.
/ 성...
문서 5: 190 ► 액체를 기체로 상태 변화시키는데 소 
요되는 열은?
가. 현열 나. 감열
사. 잠열 아. 액체열
/ 잠열 : 어떤 물질 단위량을 액체로부터 같은 온도의 
기체로 변화시키는 데 필요한 열량
191 ►어떤 물체가 대기 중에서는 6【kgfl이 
고 수중에서는 4[...
문서 6: code.
가. electricity 나. engine
시’. weight 아. velocity
22a In accordance with the MARPOL 
Convention as amended, ( ) is defined 
as the process whereby...
문서 7: 최신 문졔 09 회
은 것은?
가. 내부 에너지는 일만의 함수이다.
나. 내부 에너지는 온도만이 함수이다.
사. 내부 에너지는 압력만의 함수이다. 
아. 내부 에너지는 비열만의 함수이다.
16. 이‘’C]의 물 1이kg]과 40[℃]의 물 10[kg] 
올 외부와의 열...
------------------------------
--- [최종 답변] ---
## Final Answer:
**Final Answer:** 사. 24시간
**해설:** 냉동톤은 0℃의 물 1톤을 24시간 동안에 0℃의 얼음으로 만드는 데 필요한 시간당 열량으로 1 냉동톤은 3,320 [kcal/h] 이다. 따라서, 1냉동톤은 24시간 동안 0℃의 물 1톤을 0℃의 얼음으로 만드는 냉동능력이다.

---


2장-201번. 저항 계산 문제. 정답(34초), 특수문자 Ω. 리트리버 O, 해설 O

[6단계] RAG 체인 실행
💬 질문: 90[Ω] 저항 3개를 서로 접속하여 얻을 수 있는 가장 작은 저항값은? 가. 90[Ω] 나. 15[Ω] 사. 30[Ω] 아. 60[Ω]

--- [검색된 문서 (Context)] ---
문서 1: 최신 문제 08회
가. 1. 0. 0, o 나. o, 1, o, o 19■ 다음과 같은 단순 커패시턴스 브리지회로에서
사. 0, 0, 1, 0 아. 0, 0, 0, 1
16■전압 조정기 회로에서 부하저항 rl 양단 
에 걸리는 전압은 약 몇 M인가?
으 人 R
>. ~...
문서 2: 200 ► 릴리프 밸브의 설명 중 맞는 것은?
가. 유압을 일정치로 유지한다.
나. 유압을 일정치 이상으로 유지한다.
人>. 유체의 방향을 제어한다.
아. 유량을 조정한다.
/ 릴리프 밸브는 압력이 설정 압력에 도달하면 
유체를 우회시켜 압력을 일정치로 유지한다.
20...
문서 3: 이동하면 원판이 같은 방향으로 회전한다. 이것 
은 자석의 아동에 의해 원판상에 발생하는 와전류 
와 자속과의 사이의 전자력에 의해 토크를 발생하 
는 것으로, 유도 전동기는 이 원리를 이용하여 
자석을 움직이는 대신 회전 자계를 주어서 회전 
토크를 얻는 것이다.
0...
문서 4: 최신 문제 10회
‘0’을 가리킨다면 R4의 저항은 몇 [kQ]인가?
가. 4[kQ] 나. 6[kQ]
사. 8[kQ] 아. 10[kQ]
가. 6.2[kW] 나. 8.8[kW]
사. 12.2[kW] 아. 16.2[kW]
07•변압기에서 권선비와 같은 값을 갖는 것
은?
...
문서 5: 최신 문제
나눈 값
나. 선체의 전진속도를 반류의 속도로 나눈 값 
사. 반류의 속력을 추진기의 속력으로 나눈 값 
아. 추진기의 속력을 반류의 속력으로 나눈 값
23. 배의 조파저항에 가장 큰 영향을 주는 것은?
가. Euler수 나. Froude수
사. Weber수...
문서 6: 이 필요한가? （단, 한 사람은 평균 3이 kgfl 의 힘으 
로 밀거나 끌 수 있다고 한다.）
가. 3명 나. 6명
人E 9명 아. 27명
/ 마찰계수가 0.2 이므로 800 x 0.2 / 30 = 5.48 
이므로 최소 6명이다.
045 ► 427[PS]의 원동기가...
문서 7: 제 9 회 [ 기관 m ]
01. 다음과 같은 휘트스톤 브리지(Wheatstone 
bridge) 회로에서 저항 Rx의 값은? (단, R, 
=1이⑵, R2=2이⑵, R3=3이이이고 검류 
계 G에는 전류가 흐르지 않는 상태이다.) 
가. 2이이 나. 4이이
사. 5(...
------------------------------
--- [최종 답변] ---
## Final Answer:
**Final Answer :** 사. 30[Ω]

**해설:**
문제에서 90[Ω] 저항 3개를 서로 접속하여 얻을 수 있는 가장 작은 저항값을 묻고 있습니다. 저항을 직렬로 연결하면 저항값은 더 커지고, 병렬로 연결하면 저항값은 더 작아집니다. 따라서 3개의 저항을 병렬로 연결하면 가장 작은 저항값을 얻을 수 있습니다. 3개의 저항을 병렬로 연결하면, 각 저항의 저항값은 90[Ω] / 3 = 30[Ω]이 됩니다. 따라서 3개의 저항을 병렬로 연결했을 때 얻어지는 가장 작은 저항값은 30[Ω]입니다.

---


3장-74번. 한영 혼합 문제. 정답(24초), 리트리버 O, 해설

[6단계] RAG 체인 실행
💬 질문: 초음파 유속계는 어떤 효과를 이용한 것인가? 가. 제벡 효과 (Seebeck effect) 나. 압전 효과 사. 홀 효과 (Hall effect) 아. 도플러 효과 (Doppler effect)

--- [검색된 문서 (Context)] ---
문서 1: 펄스가 얻어진다.
073 ► 다이오드 여러 개를 직렬로 연결하면?
가. 더 높은 역방향전압에 견딜 수 있다. 
나. 더 많은 부하전류를 흘릴 수 있다. 
사. 맥동이 더 작은 정류파형을 얻을 수 있다. 
아. 순방향전압 크기를 감소시킬 수 있다.
/ 다이오드는 역방향 ...
문서 2: 전류를 흘린다.
102► 교류회로에서 유효전력이 3[w], 피 
상전력이 5【VA]일 때 역률은 얼마인가?
가. 0.5 나. 0.6 人?. 0.7 아. 0.8
/ 역률 = 유효전력 / 피상전력
103 ► 다이오드 여러 개를 직렬로 연결하면?
가. 더 높은 역방향전압에 ...
문서 3: 가. 벨로스 나. 다이어프램 
사. 노즐과 플래퍼 아. 대시포트
/ 노즐과 플래퍼 : 미세한 변위를 공기압신호로 
변환하는 기구
066 ► 다음 중 열전대와 관계가 깊은 것은?
가. 제벡 효고KSeebeck effect)를 이용한다. 
나. 압전 효과를 이용한다.
사....
문서 4: 지를 반복하는 데 필요한 검출기는?
가. 레벨 스위치 나. 온도 스위치
사. 압력 스위치 아. 리밋 스위치
/ 압력스위치에서 압력이 낮아지면 압축기를 기동 
시켜 압력탱크에 공기를 채우고 압력이 높아지면 
압축기가 멈추도록 한다.
198 ► 유압이 일정압력 이상이 되었...
문서 5: 경우에는 일반 다이오드와 같은 특성을 가지며, 
역방향전압일 경우에는 일반적으로 전류가 흐르 
도록 하는 장치
142 ► 광전센서의 장점이 아닌 것은?
가. 자계의 영향을 받지 않는다.
나. 수명이 길다.
사. 주위의 광노이즈에 강하다.
아. 검출거리가 비교적 길다
/...
문서 6: 02. 평상시에 이상없이 작동하던 유도전동기가 기 
동되지 않는다면 가장 우선적으로 점검해야 
할 것은?
가. 전압 측정 나. 주파수 측정 
사. 절연저항 측정 아. 전류 측정
03.1 이！同의 정전용량을 가진 축전기(콘덴서) 두 
개를 직렬 접속하면 합성 정전용량은 ...
문서 7: 흐
드
가. 열전대 나. 피토관
사. 바이메탈 아. 서미스터
17■어떤 물리량을 수량적으로 표시하려면 그 
양과 같은 종류의 기준이 필요한데 이 비교기 
준으로 사용되는 일정한 크기의 양을 무엇이 
라 하는가?
가. 오차 나. 측정 사. 단위 아. 보정
18■ 배전반의...
------------------------------
--- [최종 답변] ---
## Final Answer Format:
**Final Answer :** 아. 도플러 효과

**해설:**
질문은 초음파 유속계가 어떤 효과를 이용하는지 묻고 있습니다. Context에서 초음파 유속계에 대한 설명은 "도플러 효과 : 파동을 발생시키는 파원과 그 파동을 관측하는 관측자 중 하나 이상이 운동하고 있을 때 발생하는 효과로, 파원과 관측자 사이의 거리가 좁아질 때에는 파동의 주파수가 더 높게, 거리가 멀어질 때에는 파동의 주파수가 더 낮게"라고 설명하고 있습니다. 따라서 초음파 유속계는 도플러 효과를 이용합니다.

---


4장-226번. 한영 + 슷자 혼합. 오답(37초), 리트리버 O, 해설 X

[6단계] RAG 체인 실행
💬 질문: 해양환경관리법에 의한 기름오염방지설비 중 기름받이의 기준으로 옳지 않은 것은? 가. 재질은 금속제일 것 나. 선박이 세로방향으로 10도 또는 가로방향으로 22.5도 경사하는 경우에도 기름이 넘치거나 흐르지 않는 구조일 것 사. 기름이 새어나올 우려가 있는 각종 기기나 탱크(tank)의 밑에 설치 아. 모든 탱크의 에어벤트(tank air vent)에는 반드시 기름 받이가 있을 것

--- [검색된 문서 (Context)] ---
문서 1: 비 중 기름받이의 기준으로 옳지 않은 것은? 
가. 재질은 금속제일 것
나. 선박이 세로방향으로 10도 또는 가로방향 
으로 22.5도 경사하는 경우에도 기름이 
넘치거나 흐르지 않는 구조일 것
사. 기름이 새어나올 우려가 있는 각종 기기 
나 탱크(tank)의 밑에 ...
문서 2: 직무일반 척중 예상 묜졔
가 수면위로 올라올 가능성이 있어 선미를 무겁게 
하기 위해 배수하여서는 안된다.
223 ► 연료유 저장탱크의 점검차 탱크 내부에 
들어가는 경우의 주의사항 중 옳지 않은 것은?
가. 송풍장치로 탱크 내부의 가스를 제거한 
후 들어갈 것
나. ...
문서 3: 217 ► 다음 중 기름의 해양배출이 허용되는 경 
우는?
가. 항해 중 15[ppm] 이상의 유분농도의 
처리수로 배출할 경우
나. 영해기선으로부터 25해리 이상에서 항 
해 중에 기관실 빌지펌프를 통해 배출하 
는 경우
사. 기관실 기기를 수리한 후 발생한 폐유를 ...
문서 4: s
아. 기름 탱크 속에서 작업을 할 때에는 미리 
충분히 환기시켜야 한다.
/ 몸은 전류가 흐르는 도체 이므로 옷을 길게 입어 
전기가 통하지 않도록 한다.
162 ► 다음 중 기름 또는 유해액체물질의 해양 
오염비상계획서가 비치되어야 할 선박이 아 
닌 것은?
가....
문서 5: 나. 기름유출사고 발생 시 해당 선박의 기관장 
이 취해야 할 보고 절차에 관한 사항
人1. 선원의 방제교육 • 훈련에 관한 사항 
아. 기름유출을 줄이기 위하여 해당 선박의 
선원이 취할 방제조치에 관한 사항
/ 선박에서의 오염방지에 관한 규칙 제25조（선박 
해양오...
문서 6: 부터 제47조까지 및 제47조의2는 다음 각 호의 
어느 하나에 해당하는 경우에는 적용하지 아니한다.
1. 선박 및 해양사설의 안전 확보 또는 인명구조를 
위하여 부득이하게 대기오염물질이 배출되는 
경우
2. 선박 또는 해양시설의 손상 등으로 인하여 
부득이하게 대기오...
문서 7: 히 배출한다.
사. 자동제어기기와 원격제어기기의 작동상 
태를 재확인한다.
아. 추진기관의 공회전을 피하기 위해 선미탱 
크를 배수한다.
04. 피복아크 용접에 대한 재해방지 대책으로 옳 
지 않은 것은?
가. 눈을 보호하기 위해 차광보호면을 사용 
한다.
나. 용접기...
------------------------------
--- [최종 답변] ---
## Final Answer:
**Final Answer:** 사. 기름이 새어나올 우려가 있는 각종 기기나 탱크(tank)의 밑에 설치
**해설:** 해양환경관리법에 의한 기름받이의 기준은 다음과 같습니다.
1) 재질 : 금속제일 것
2) 구조 : 선박이 세로방향으로 10도 또는 가로방향으로 22.5도 경사하는 경우에도 기름이 넘치거나 흐르지 않는 구조일 것
3) 설치장소 : 유탱크,유면계,유펌프,여과기,보일러화구,유청정기,부산밸브시험장치, 주기관, 보조기관, 기름이 흐르는 관장치의 밸브 또는 콕 그 밖에 기름이 새어나올 우려가 있는 장치의 밑에 설치할 것
따라서, 기름이 새어나올 우려가 있는 각종 기기나 탱크(tank)의 밑에 설치하는 것은 기름받이 기준에 해당하지 않습니다.

---



*랜덤 문제 정답률
5/8 = 0.625

*bge-m3 임베딩딩
리트리버 O

*Gemma-3-n
한국어 추론 및 해설 X, 숫자 계산 O-> 모델 수정 필요